So let's just have make sure that we have a well written and well documented code. So please explain everything you do and have the pevious (working version) of the program stored somewhere incase of something does not work after you alter it's implementation. Also, for reference, feel free to add resources when needed.

In [ ]:
# state all imports over here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import sklearn
from sklearn.metrics import *
from sklearn.decomposition import *

In [ ]:
# importing our dataset
client_info = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv')
client_record = pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv')

1. Data Pre-processing

In [ ]:
# importing our dataset
client_info = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv')

client_record = pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv')
'''
Since the data we are using has alot of users than the number of transactions, 
we will only consider users who have made a transaction recently
first we will read the datasets into pandas dataframes
'''


''' dropping occupation type as it has a lot of missing values'''
# print(client_info['OCCUPATION_TYPE'].unique())
client_info = client_info.drop(['OCCUPATION_TYPE'],axis=1)
# print(client_info.columns)


'''for mapping gender values
we have 1 for Male and 0 for Female'''
client_info['CODE_GENDER'].replace({"M": 1, "F": 0}, inplace=True)
    

'''for mapping whether the client has a car
it will be 1 for yes and 0 for no'''
client_info["FLAG_OWN_CAR"].replace({"Y": 1, "N": 0}, inplace=True)


'''for mappping whether the client owns any real esate assents
again it will be 1 for 1 for yes and 0 for nonlocal'''
client_info["FLAG_OWN_REALTY"].replace({"Y": 1, "N": 0}, inplace=True)    


'''for mapping educational qualifications
we have not made dummy varaibles for this because we feel
higher educatonal qualifications should be given higher weight'''
client_info["NAME_EDUCATION_TYPE"].replace({"Higher education": 3, "Lower secondary": 0, "Secondary / secondary special": 1, "Incomplete higher": 2, "Academic degree":4}, inplace=True)


'''for mapping housing type
the value 0 would indicate a rented/municipal/office apartment 
or that the client lives with his parents whereas 1 would indicate a House/co-op apartment
the reason being that 1 would indicate that the person lives on a property that they own 
whereas 0 would indicate that it is not a permanent place'''
client_info["NAME_HOUSING_TYPE"].replace({"House / apartment":5, 'Co-op apartment':4, 'Municipal apartment':3, 'With parents':2, 'Office apartment':1, 'Rented apartment':0}, inplace=True)


'''for mapping income type
the value 0 would indicate a student/person living on pension 
whereas the value 1 would indicate a commercial associate or a person who 
is working or is a state servent. We have done so because a pensioner and 
a student are people who are not actively working and hence should be considered
into a different category from the people who are actively working'''
client_info['NAME_INCOME_TYPE'].replace({'Working':3, 'Commercial associate':4, 'Pensioner':1, 'State servant':2, 'Student':0}, inplace=True)


'''for mapping family status
we will classify marriage and civil marriage as 1 since they both indicate
that the person currently has a spouse and we will classify separated, single
and widow as 0 since in all of them the client has no spouse'''
client_info['NAME_FAMILY_STATUS'].replace({'Single / not married':2, 'Separated':1, 'Widow':0, 'Married':4, 'Civil marriage':3}, inplace=True)


''' for handling our response variable
NOTE THIS IS SUBJECT TO CHANGE
so the variable STATUS will be our response variable
for loans that were paid as safe credit approvals whereas all
dues (all integer values) as unsafe credit approvals. The values assigned would be
1 and 0 respectively. We might have to change this depending on accuracy'''
client_record['STATUS'].replace({'C':1, 'X':1,'1':0,'0':0,'2':0,'3':0,'4':0,'5':0},inplace=True)


# '''for handling occupation type'''
# client_info['OCCUPATION_TYPE'].replace({'Security staff':4, 'Sales staff':7, 'Accountants':16, 'Laborers':2, 'Managers':17, 'Drivers':6, 'Core staff':9, 'High skill tech staff':18, 'Cleaning staff':3, 'Private service staff':10, 'Cooking staff':5, 'Low-skill Laborers':1, 'Medicine staff':15, 'Secretaries':12, 'Waiters/barmen staff':8, 'HR staff':11, 'Realty agents':14, 'IT staff':13},inplace=True)


'''One hot encoding'''
#this was previously client_info1
client_info = pd.get_dummies(client_info, columns = ["NAME_EDUCATION_TYPE", "NAME_HOUSING_TYPE", "NAME_INCOME_TYPE", "NAME_FAMILY_STATUS"], drop_first=True)
# print(client_info1.head())

''' Merging the 2 datasets'''
data = client_record.merge(client_info, how = 'left', on = 'ID')
data = data.dropna()
data = data.reset_index()
data = data.drop(columns=['index'])
data = data.set_index('ID')
data = data.astype('float64')
# data = abs(data)
tempo = data['STATUS']
# print(tempo)
# print(len(data))
# print(data["NAME_INCOME_TYPE"].dtype)
# print(client_record.dtypes)


'''To normalize the dataset we used 2 differenet approaches
1. Mean normalization
2. Min-Max normalization '''
data_mean_norm = (data-data.mean())/data.std()
data_min_max_norm = (data-data.min())/(data.max()-data.min())


'''since flag_mobil has all column entries as 1, it gives NaN value
upon normalization. To ensure no loss of data we are reassigning 
it its original value'''
# data['FLAG_MOBIL'].unique(); proof for the above statement
data_mean_norm['FLAG_MOBIL'] = 1.0
data_min_max_norm['FLAG_MOBIL'] = 1.0
data_mean_norm['STATUS'] = tempo
data_min_max_norm['STATUS'] = tempo
# print(data_mean_norm.head())
# print(data_min_max_norm.head())
# client_info1.columns

In [ ]:
y1=data_min_max_norm['STATUS']
y1k = copy.deepcopy(y1)
y1=y1.to_numpy()
X1=copy.deepcopy(data_min_max_norm)
X1=X1.drop(['STATUS','CNT_CHILDREN','DAYS_BIRTH'],axis=1)
# X1=X1.drop(['STATUS'],axis=1)
X1k = copy.deepcopy(X1)
X1=X1.to_numpy()
ones=np.ones((X1.shape[0],1))
X1 = np.concatenate((ones, X1), axis=1)

In [ ]:
y2=data_mean_norm['STATUS']
y2k = copy.deepcopy(y2)
y2=y2.to_numpy()
X2=copy.deepcopy(data_mean_norm)
X2=X2.drop(['STATUS','CNT_CHILDREN','DAYS_BIRTH'],axis=1)
# X1=X1.drop(['STATUS'],axis=1)
X2k = copy.deepcopy(X2)
X2=X2.to_numpy()
ones=np.ones((X2.shape[0],1))
X2 = np.concatenate((ones, X2), axis=1)

Implimenting K folds; to divide data into trainning and testing data
The function "split" take 3 inputs X:Input nummpy array
y:nummy array (i.e our status)
ratio: the splitting ratio

In [ ]:
''' BOX PLOT FOR ALL FEATURES'''
print(data.columns)
# print(data_mean_norm.boxplot())
print(data_min_max_norm.plot(kind='box', rot=75))

In [ ]:
pca = PCA(n_components=1)
xn = copy.deepcopy(data_min_max_norm)
xxn = data_min_max_norm['STATUS']
xxn = xxn.reset_index()
# print(xxn)
xn = xn.drop(['STATUS'],axis=1)
tdata = pca.fit_transform(xn)
tt = pd.DataFrame(tdata)
# print(tt)
tt['STATUS'] = xxn['STATUS']
print(tt.columns)
print(tt.boxplot()) 

In [ ]:
# removing outliers
Q1 = tt[0].quantile(0.25)
Q3 = tt[0].quantile(0.75)
IQR = Q3 - Q1  #IQR is interquartile range. 
norm_pts = ( tt[0] >= Q1 - 1.5 * IQR) & ( tt[0] <= Q3 + 1.5 *IQR)
norm_pts.value_counts()


In [ ]:
# heat map for correlation
import seaborn as sns
print(data.columns)
corr = data.corr()
plt.figure(figsize=(15,7.5))
ax = sns.heatmap(corr, square=True)
print(ax)


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


def Mylogestic(X_train,X_test,y_train,y_test):
    
    clf = LogisticRegression(random_state=10, max_iter=10000).fit(X_train, y_train)
    return(clf.predict(X_test))


def train_test(X,y,ratio):
    kf = KFold(n_splits=ratio)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        fin=Mylogestic(X_train,X_test,y_train,y_test)
        print(accuracy_score(y_test, fin))
        print(confusion_matrix(y_test, fin))
    return 

Simple Logestic Regression function (input should be nummy array).Returns our predicted value

THE CONFUSION MATRIX

TN,FP

FN,TP

In [ ]:
#for min max
train_test(X1,y1,10)

In [ ]:
#for mean
train_test(X2,y2,10)

In [ ]:
data.columns
# data.corr()   